In [11]:
import sys
sys.path.insert(1, 'Module')
import time

"##### Start programm with run #####"
Input_File = True

start_timer = time.perf_counter()
def print_start(name):
    print(f'Start Simulation {name}')
if Input_File:
    from read_input_excel import *
else:
    from alternative_input import *

if __name__ == '__main__':
    print_start('Gesamtkonzept Strom')

%run ./Module/Simulation_Komponenten.ipynb

end_timer = time.perf_counter()
running_time_min = round((end_timer-start_timer)/60,2)
print("Programm fertig: Laufzeit = "+str(running_time_min)+" MIN")
quit()

Start Simulation Gesamtkonzept Strom
Simulation Running
Simulation ELKW
Analyse für monatliche Spitze
                     Time  kW_netzbezug  dach_süd  dach_ostwest  carport  \
0     2022-01-01 00:00:00    254.118331         0           0.0      0.0   
1     2022-01-01 00:15:00    254.118331         0           0.0      0.0   
2     2022-01-01 00:30:00    254.118331         0           0.0      0.0   
3     2022-01-01 00:45:00    251.414944         0           0.0      0.0   
4     2022-01-01 01:00:00    243.304785         0           0.0      0.0   
...                   ...           ...       ...           ...      ...   
35035 2022-12-31 22:45:00    135.169325         0           0.0      0.0   
35036 2022-12-31 23:00:00    145.982871         0           0.0      0.0   
35037 2022-12-31 23:15:00    137.872711         0           0.0      0.0   
35038 2022-12-31 23:30:00    135.169325         0           0.0      0.0   
35039 2022-12-31 23:45:00    145.982871         0           0.

ValueError: too many values to unpack (expected 2)

ValueError: too many values to unpack (expected 2)